# LOAD ALL FILES FROM FILES_FOR_DEFINITIVE_TEST_CLAP_EMBEDDINGS IN /CONTENT/

In [ ]:
# CHECK THAT THE NUMPY VERSION IS 1.26.4

import numpy as np
print(np.__version__)

#otherwise install it
#!pip install --force-reinstall numpy==1.26.4

#note - the results vary if a more recent version of numpy is utilized instead of that used during the project development

1.26.4


Download the required files and dictionaries (you can use gdown or any library you prefer)

-> https://drive.google.com/file/d/1hvXfqOrVL1igzMOe31soZ46SnVjm5rg-/view?usp=sharing

In [ ]:
#@markdown #LOADING FILES

import pickle
total_nodes = []
all_hinormer_node_embeddings = {}
all_hinormer_audio_embeddings = {}
all_relphormer_node_embeddings = {}
all_relphormer_audio_embeddings = {}
all_relphormer_vanilla_node_embeddings = {}

with open('/content/biggest_hinormer_total_nodes.pkl', 'rb') as f:
    total_nodes = pickle.load(f)

with open('/content/relphormer_biggest_all_node_embeddings_v2.pkl', 'rb') as f:
    all_relphormer_node_embeddings = pickle.load(f)

with open('/content/relphormer_biggest_all_audio_embeddings_v2.pkl', 'rb') as f:
    all_relphormer_audio_embeddings = pickle.load(f)

with open('/content/hinormer_biggest_all_node_embeddings.pkl', 'rb') as f:
    all_hinormer_node_embeddings = pickle.load(f)

with open('/content/hinormer_biggest_all_audio_embeddings.pkl', 'rb') as f:
    all_hinormer_audio_embeddings = pickle.load(f)

with open('/content/relphormer_biggest_vanilla_all_node_embeddings.pkl', 'rb') as f:
    all_relphormer_vanilla_node_embeddings = pickle.load(f)

import pickle
biggest_degree_centrality_for_uri = {}
biggest_pagerank_for_uri = {}
biggest_uri_to_popularity = {}

with open('/content/biggest_degree_centrality_for_uri.pkl', 'rb') as f:
    biggest_degree_centrality_for_uri = pickle.load(f)

with open('/content/biggest_pagerank_for_uri.pkl', 'rb') as f:
    biggest_pagerank_for_uri = pickle.load(f)

with open('/content/biggest_uri_to_popularity.pkl', 'rb') as f:
    biggest_uri_to_popularity = pickle.load(f)

import pickle


with open('/content/old_id_to_new_id.pkl', 'rb') as f:
    old_id_to_new_id = pickle.load(f)

with open('/content/new_id_to_old_id.pkl', 'rb') as f:
    new_id_to_old_id = pickle.load(f)

with open('/content/old_uri_to_new_id.pkl', 'rb') as f:
    old_uri_to_new_id = pickle.load(f)

In [ ]:
#@markdown #DEFINE THE FETCH_RECOMMENDATIONS FUNCTION

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Number_of_Recommendations_per_Model = 100 #@param {type:"number"}

def find_top_10_audio(node_tensor, all_audio_embeddings):

    audio_keys = list(all_audio_embeddings.keys())
    audio_embeddings = np.stack([all_audio_embeddings[key] for key in audio_keys])
    similarities = cosine_similarity(audio_embeddings, [node_tensor]) #does cosine similarity with all the embeddings of the opposite media
    top_10_indices = np.argsort(np.squeeze(similarities, axis=1))[-Number_of_Recommendations_per_Model:][::-1]
    top_10_audio_keys = [audio_keys[idx] for idx in top_10_indices]
    return top_10_audio_keys, np.squeeze(similarities, axis=1)[top_10_indices]

def find_top_10_nodes(audio_tensor, all_node_embeddings):

    node_keys = list(all_node_embeddings.keys())
    node_embeddings = np.stack([all_node_embeddings[key] for key in node_keys])
    similarities = cosine_similarity(node_embeddings, [audio_tensor])
    top_10_indices = np.argsort(np.squeeze(similarities, axis=1))[-(5*Number_of_Recommendations_per_Model):][::-1]
    top_10_node_keys = [node_keys[idx] for idx in top_10_indices]
    return top_10_node_keys, np.squeeze(similarities, axis=1)[top_10_indices]

In [ ]:
#@markdown #CREATE LISTS AND DICTIONARIES

import pandas as pd

orig_tracks_dataset = pd.read_csv("/content/nva_final_tracks_less.csv")
new_tracks_dataset = pd.read_csv("/content/more_tracks_less_modified.csv")
tracks_dataset = orig_tracks_dataset.merge(new_tracks_dataset,how="outer")
playlists_dataset = pd.read_csv("/content/final_playlists.csv")
artists_dataset = pd.read_csv("/content/artists.csv")
nomi_tracce = tracks_dataset['name'].values
data_tracce = tracks_dataset['release_date'].values
uri_tracce = tracks_dataset['track_uri'].values
track_indexes_for_playlists = {}
track_indexes_for_artists = {}
total_artists = []
total_playlists = []
total_titles = []

for i, row in tracks_dataset.iterrows():
  nome_traccia = row['name']
  nome_artisti = row['artists_names']
  playlist_uris = row['playlist_uris']
  nome_artisti = nome_artisti.strip('][').strip("'").strip('"').split("', '")

  for k in nome_artisti:
    if k not in track_indexes_for_artists:
      track_indexes_for_artists[k] = []
      total_artists.append(k)
    track_indexes_for_artists[k].append(i)

  nome_artisti = ", ".join(nome_artisti)
  playlist_uris = playlist_uris.strip('][').strip("'").split("', '")
  total_titles.append(nome_artisti+" - "+nome_traccia)

old_track_uris = orig_tracks_dataset['track_uri']
new_track_uris = new_tracks_dataset['track_uri']

orig_order_track_uris = list(np.concatenate((old_track_uris,new_track_uris))) #crea una lista con l'ordine corretto degli audio files

In [ ]:
#@markdown #MAIN FUNCTION FOR INDEX REBALANCING

import numpy as np
from itertools import islice


def remove_duplicates(audio_tracks, probability_indexes): #remove duplicate files (mostly used on audio2node coz of the 5 equal embeddings for each node -> 0_1, 0_2, 0_3... have the same node embedding)
    unique_entries = {}

    for track, prob in zip(audio_tracks, probability_indexes):
        prefix = track.split('_')[0]

        if (prefix, prob) not in unique_entries:
            unique_entries[(prefix, prob)] = track

    final_audio_tracks = np.array(list(unique_entries.values()))
    final_probability_indexes = np.array([key[1] for key in unique_entries.keys()])

    return final_audio_tracks, final_probability_indexes


def normalize_and_weight_similarity(indexes, search_type):
    min_val = np.min(indexes)
    max_val = np.max(indexes)
    normalized_indexes = (indexes - min_val) / (max_val - min_val + 1e-8) #map to interval 0-1

    mapped_indexes = 0.7 * normalized_indexes + 0.3

    return mapped_indexes

def process_and_normalize_results(temp_audio_results, search_type):
    results_keys = np.array(list(temp_audio_results.keys()))
    results_scores = np.array(list(temp_audio_results.values()))
    normalized_scores = normalize_and_weight_similarity(results_scores, search_type)
    normalized_results = {key: score for key, score in zip(results_keys, normalized_scores)}
    return normalized_results


def find_similarities(new_num, search_type, embeddings_1, embeddings_2, all_similarities_dictionary, displaytext, top_results_dictionaries):
  # search types can be: relphormer_node, hinormer_node, relphormer_audio, hinormer_audio or node_to_node

  int_new_num = int(new_num[:-7])

  temp_audio_results = {}
  current_top_results_keys = []
  current_top_results_scores = []

  if search_type == "relphormer_node" or search_type == "hinormer_node":

    raw_temp_audio_results = {}

    for h in range(5):
      results, indexes = find_top_10_nodes(embeddings_1[new_num[:-7]+"_"+str(h)],embeddings_2)
      results, indexes = remove_duplicates(results, indexes)

      for ix,i in enumerate(results):

        if i in raw_temp_audio_results.keys():
          raw_temp_audio_results[i] = raw_temp_audio_results[i] + indexes[ix]
        else:
          raw_temp_audio_results[i] = indexes[ix]

        if ix < 100: #this limit is to avoid people putting numbers too high because the performance gets a lot worse, you can override
          current_top_results_keys.append(i)
          current_top_results_scores.append(indexes[ix])

    temp_audio_results = process_and_normalize_results(raw_temp_audio_results, search_type)

    sorted_audio_results = {k: v for k, v in sorted(temp_audio_results.items(), key=lambda x: x[1], reverse=True)}

    for k,v in sorted_audio_results.items():

      if k in all_similarities_dictionary.keys():
            all_similarities_dictionary[k] = all_similarities_dictionary[k] + v
      else:
          all_similarities_dictionary[k] = v


  elif search_type == "relphormer_audio" or search_type == "hinormer_audio":
    results, indexes = find_top_10_audio(embeddings_1[new_num[:-5]],embeddings_2)
    results, indexes = remove_duplicates(results, indexes)
    indexes = normalize_and_weight_similarity(indexes, search_type)

    for ix,i in enumerate(results):

      if i in all_similarities_dictionary.keys():

        all_similarities_dictionary[i] = all_similarities_dictionary[i] + indexes[ix]
      else:
        all_similarities_dictionary[i] = indexes[ix]

      if ix < 100:
        current_top_results_keys.append(i)
        current_top_results_scores.append(indexes[ix])

  elif search_type == "node_to_node": # relphormer node to node comparison

    results, indexes = find_top_10_nodes(embeddings_1[new_num[:-5]],embeddings_2)
    results, indexes = remove_duplicates(results, indexes)

    indexes = indexes / 2 #this one is naturally weighted down a bit

    for ix,i in enumerate(results):


      if i in all_similarities_dictionary.keys():
        all_similarities_dictionary[i] = all_similarities_dictionary[i] + indexes[ix]
      else:
        all_similarities_dictionary[i] = indexes[ix]

      if ix < 100:
        current_top_results_keys.append(i)
        current_top_results_scores.append(indexes[ix])

  else: print("search type unknown")

  #print(f"original {displaytext} results: \n\n,{results}") #they are wrong anyway

  top_results_dictionaries[search_type] = {"keys": current_top_results_keys, "scores": current_top_results_scores}

  return all_similarities_dictionary, top_results_dictionaries

def find_similar_tracks(input_file):

  all_similarities_dictionary = {}
  top_results_dictionaries = {}

  #do all 5 different embedding comparison tasks
  all_similarities_dictionary, top_results_dictionaries = find_similarities(input_file, "relphormer_audio", all_relphormer_node_embeddings, all_relphormer_audio_embeddings, all_similarities_dictionary, "relphormer node to audio", top_results_dictionaries)
  all_similarities_dictionary, top_results_dictionaries = find_similarities(input_file, "relphormer_node", all_relphormer_audio_embeddings, all_relphormer_node_embeddings, all_similarities_dictionary, "relphormer audio to nodes", top_results_dictionaries)
  all_similarities_dictionary, top_results_dictionaries = find_similarities(input_file, "hinormer_audio", all_hinormer_node_embeddings, all_hinormer_audio_embeddings, all_similarities_dictionary, "hinormer node to audio", top_results_dictionaries)
  all_similarities_dictionary, top_results_dictionaries = find_similarities(input_file, "hinormer_node", all_hinormer_audio_embeddings, all_hinormer_node_embeddings, all_similarities_dictionary, "hinormer audio to nodes", top_results_dictionaries)
  all_similarities_dictionary, top_results_dictionaries = find_similarities(input_file, "node_to_node", all_relphormer_vanilla_node_embeddings, all_relphormer_vanilla_node_embeddings, all_similarities_dictionary, "relphoremer vanilla node to node", top_results_dictionaries)

  final_weighted_similarities = {}
  for track_key, initial_score in all_similarities_dictionary.items():
      num_appearances = 0
      position_bias = 0.0
      for search_type in top_results_dictionaries:
          if track_key in top_results_dictionaries[search_type]["keys"]:
              num_appearances += 1
              rank = top_results_dictionaries[search_type]["keys"].index(track_key) + 1

      if num_appearances > 1: #if a song appears more than once, it should get a small boost
          for search_type in top_results_dictionaries:
              if track_key in top_results_dictionaries[search_type]["keys"]:
                  rank = top_results_dictionaries[search_type]["keys"].index(track_key) + 1
                  position_bias += 0.3 * np.exp(-0.1 * (rank - 1)) #calculate position bias (the higher the track appears on multiple models, the more is relevant)

      base_weight = 1.0 if num_appearances > 1 else 0.7 #weight down if appears once
      weighted_score = initial_score * base_weight * (num_appearances / 2.0 if num_appearances > 0 else 0.5) + position_bias #custom made formula (quite random)
      final_weighted_similarities[track_key] = weighted_score


  print("\n\n\n",total_titles[old_uri_to_new_id[orig_order_track_uris[int(input_file[:-7])]]])
  print(old_uri_to_new_id[orig_order_track_uris[int(input_file[:-7])]])

  print("\n\n\n\n DICTIONARY SORTED SIMILARITIES: \n\n\n\n")
  sorted_similarities = {k: v for k, v in sorted(final_weighted_similarities.items(), key=lambda x: x[1], reverse=True)}
  for k,v in islice(sorted_similarities.items(),30):
    print(total_titles[old_uri_to_new_id[orig_order_track_uris[int(k[:-2])]]])
    print("weighted similarity: ",v)
    print("degree centrality: ",biggest_degree_centrality_for_uri[orig_order_track_uris[int(k[:-2])]])
    print("pagerank value: ",biggest_pagerank_for_uri[orig_order_track_uris[int(k[:-2])]])
    print("spotify popularity: ",biggest_uri_to_popularity[orig_order_track_uris[int(k[:-2])]])
    print("\n\n")

In [ ]:
#@markdown #HERE YOU CAN FIND THE INDEX OF A TRACK IN THE DATASET

term_to_search = 'Michael Jackson' #@param {type:"string"}

search_mask = tracks_dataset.apply(lambda row: row.astype(str).str.contains(term_to_search).any(), axis=1)
search_results = tracks_dataset.loc[search_mask]
results_uris = search_results['track_uri']

result_indexes = [old_uri_to_new_id[uri] for uri in results_uris]
actual_result_indexes = [new_id_to_old_id[index] for index in result_indexes]
for ix,i in enumerate(result_indexes):
  if actual_result_indexes[ix] > 8000: #<8000 = training set
    print(total_titles[i]," : ",actual_result_indexes[ix])

Michael Jackson - Bad (feat. Pitbull)  :  14318
Michael Jackson - Man In The Mirror  :  12683
Michael Jackson - Off the Wall  :  19275
Michael Jackson - The Way You Make Me Feel - Single Version  :  9146
Michael Jackson - Bad  :  16905
Michael Jackson - Get on the Floor  :  15161


In [ ]:
#@markdown #INSERT THE TRACK INDEX TO OBTAIN RECOMMENDATIONS

index_to_process = "16905" #@param {type:"string"}

find_similar_tracks(index_to_process+"_0.flac")




 Michael Jackson - Bad
17562




 DICTIONARY SORTED SIMILARITIES: 




Bobby O - I'm So Hot for You
weighted similarity:  3.093314280228493
degree centrality:  8.828657823294414e-05
pagerank value:  1.4481481475143374e-05
spotify popularity:  25



Amii Stewart - Knock On Wood
weighted similarity:  2.861776188776239
degree centrality:  0.0001324298673494162
pagerank value:  1.5130926346385888e-05
spotify popularity:  54



McFadden & Whitehead - Ain't No Stoppin' Us Now
weighted similarity:  2.6837823448833404
degree centrality:  0.00017657315646588828
pagerank value:  1.5970628831414143e-05
spotify popularity:  54



McFadden & Whitehead - Ain't No Stoppin' Us Now
weighted similarity:  2.5894402929776152
degree centrality:  0.00017657315646588828
pagerank value:  1.5970628831414143e-05
spotify popularity:  54



Michael Jackson - Get on the Floor
weighted similarity:  2.4808794130706966
degree centrality:  8.828657823294414e-05
pagerank value:  1.1019558962637868e-05
spotify popula